### Part 4: Human in the Loop 
LangGraphs ***Persistence*** layer supports human-in-the-loop workflows , allowing execution to pause and resume based on user feedback 

##### Key Use Cases for human-in-the-loop workflows in LLM-based applications include:
        - 🛠️ Reviewing tool calls: Humans can review, edit, or approve tool calls requested by the LLM before tool execution.
        - ✅ Validating LLM outputs: Humans can review, edit, or approve content generated by the LLM.
        - 💡 Providing context: Enable the LLM to explicitly request human input for clarification or additional details or to support multi-turn conversations.



##### Key methods/functions used to faciliate human in the loop 
- The **[interrupt](https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/#interrupt)** function facilitates feedback. 
    - When called inside a node, the graph will pause execution, and once user feedback is provided, Execution can be resumed by using a **[Command](https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/#the-command-primitive)** primative, which can be passed through invoke, stream, or async equivalent methods
    - *Interupt* does not automatically resume execution from the interruption point. Instead, they rerun the entire node where the interrupt was used. For this reason, interrupts are typically best placed at the start of a node or in a dedicated node.
    - Warning: place code with overhead (api calls, chained commands that execute other commands) AFTER the interrupt to avoid duplication, as these are re-trieggered every timne the node is resumed


#### Tool usage Note
- The LLM specified is in charge of implicitly deciding what tool to use out of the portfollio of tools specified in the list of tools binded to the llm. if you want weather, you ask for weather, if u want feedback, you specificy you want someone to provide input. There is no explicit routing of tools based on specific critera other than the model 'guessing' what tool is most relevant given the user situation

##### Start with our existing code from Part 2. We will make one change, which is to add a simple human_assistance tool accessible to the chatbot. This tool uses interrupt to receive information from a human.

In [1]:
#Import API Keys       
_root = "/home/zjc1002/Mounts/code/"

# Specifcy Anthropic model to use 
_model_id = "anthropic:claude-3-5-sonnet-20240620"

In [2]:
import sys 
from typing import Annotated, Dict 

from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt


# Custom functions
sys.path.append(_root)
from  admin.api_keys import _api_keys
from admin.sys_ops import _set_env
from colab_projects.tutorials.langgraph.tools import get_weather

# This anthropic and internet tool requires  API key to be set as a environment variable(s)
for api_key in ["TAVILY_API_KEY","ANTHROPIC_API_KEY"]:
    _set_env(api_key
            ,value =  _api_keys[api_key])

In [9]:
#basic state
class State(TypedDict):
    """State for the LangGraph conversation flow.
    
    This TypedDict represents the conversation state maintained throughout 
    the graph execution, containing the message history and supporting
    human-in-the-loop interactions.
    """
    messages: Annotated[list, add_messages]


@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]

def chatbot(state: State) -> Command:
    """Process user inputs and generate AI responses in the conversation flow.
    
    This function handles the core chatbot functionality by invoking the LLM with 
    the current conversation context. It ensures proper tool handling by limiting
    to one tool call at a time to support the human-in-the-loop workflow.
    
    Args:
        state (State): The current conversation state containing message history
        llm_with_tools (ChatAnthropic): The language model with bound tools
        
    Returns:
        Command: Dictionary containing the AI response message
    """
    # Process the conversation state and generate a response
    message = llm_with_tools.invoke(state["messages"])
    assert(len(message.tool_calls) <= 1)
    return {"messages": [message]}


def stream_graph_updates(user_input:str,graph ,  config:Dict): 
    """Stream updates from a LangGraph execution and print the responses.
    
    This function processes a user input through a LangGraph conversation flow,
    streaming the results as they become available and printing each message.
    
    Args:
        user_input (str): The user's question or instruction to process
        graph (CompiledStateGraph): The compiled LangGraph object that defines the conversation flow
        config (Dict): Configuration settings for the graph execution, including thread ID
    """
    events = graph.stream(
        {'messages': [{'role': "user", "content": user_input}]}
         , config
         , stream_mode = 'values'
         )
    
    for event in events: 
        event['messages'][-1].pretty_print()
    


def stream_graph_updates_human_input(human_response:str
                                     , graph 
                                     ,  config:Dict): 
    """
    Stream updates from a LangGraph execution and print the responses.
    """
    
    events = graph.stream(
        (human_response)
         , config
         , stream_mode = 'values'
         )
    
    for event in events: 
        event['messages'][-1].pretty_print()
    

#specify tools 
internet_search = TavilySearch(max_results=2)
tools = [internet_search, human_assistance, get_weather]

#compile tools and llm 
llm = init_chat_model(_model_id) #generalized method to load any supported decoder for use as chatbot 
llm_with_tools = llm.bind_tools(tools)
tool_node = ToolNode(tools=tools)

######
#CREATE THE GRAPH
######

#compile the graph
graph_builder = StateGraph(State)

#add chatbot and tools to the graph
graph_builder.add_node("chatbot", chatbot)

#add tool node to the graph
graph_builder.add_node("tools", tool_node)

# TOOLS are processed and evaluated for use via conditional edges
# The tools_condition function is a custom function that determines whether the chatbot should use a tool based on the current state of the conversation.
# When add_conditional_edges is called with the "chatbot" node and this function, it's establishing the logic that determines where the conversation flow should go after the chatbot node processes a message.
# This creates a decision point in the graph where the conversation can either:Route to the tools node if the chatbot determines it needs external information or Loop back to the chatbot or go to another node if no tools are needed
# This conditional routing allows the chatbot to dynamically decide when to use the provided tools (internet search, human assistance, or weather information) based on the conversation context.
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

graph_builder.add_edge("tools", "chatbot") #generates / creates a path from tools to chatbot
graph_builder.add_edge(START, "chatbot") #defines the starting point of the graph

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)


In [ ]:
##RUN IT (NOTE: if you want to hold 2 conversations at once, you need to set the thread_id in the config)
user_input = 'What is the weather in Charlotte North Carolina?'
config = {"configurable": {"thread_id": "1"}}
stream_graph_updates(user_input, graph,  config )


#human feedback request 
user_input1 = 'I need some expert guidance on digital resources best suited to automate the planning of a day trip to charlotte. Could you please request assistance for me?'
# The chatbot generated a tool call, but then execution has been interrupted! Note that if we inspect the graph state, we see that it stopped at the tools node:
# we can see the next stage in the graph is a tool call
# NOTE: The state is loaded in the first step so that our chatbot can continue where it left off.
snapshot = graph.get_state(config)
print(snapshot.next)
stream_graph_updates(user_input1, graph,  config )


###HUMAN / EXPERT INPUT 
# To resume execution, we pass a Command object containing data expected by the tool. 
# The format of this data can be customized based on our needs. Here, we just need a dict with a key "data":
# Ask for human input instead of hardcoding the response
print("The user is asking for guidance about planning a day trip to Charlotte.")
print("Please provide your expert guidance as input:")
human_response = input()

#command to resume graph operations after human input
human_command = Command(resume={"data": human_response})
stream_graph_updates_human_input(human_command, graph,  config )

user_input2 = 'Today is April 28 2025. Given the weather in  Charlotte and the expert guidance provided by an charlotte expert,  what are some upcoming events in charlotte for the first weekend in May 2025?'
stream_graph_updates(user_input2, graph,  config )

================================ Human Message =================================

What is the weather in Charlotte North Carolina?
================================== Ai Message ==================================

[{'text': "Certainly! I can help you get the current weather information for Charlotte, North Carolina. To do this, I'll use the get_weather function, which provides weather descriptions for given cities. Let me fetch that information for you.", 'type': 'text'}, {'id': 'toolu_015VcwcDg5H3UKAKZ7TwMb3q', 'input': {'city': 'Charlotte'}, 'name': 'get_weather', 'type': 'tool_use'}]
Tool Calls:
  get_weather (toolu_015VcwcDg5H3UKAKZ7TwMb3q)
 Call ID: toolu_015VcwcDg5H3UKAKZ7TwMb3q
  Args:
    city: Charlotte
================================= Tool Message =================================
Name: get_weather

Charlotte: ☀️   +73°F
================================== Ai Message ==================================

Based on the information provided by the get_weather function, here's the c